In [2]:
import torch
import numpy as np
import utils
from sklearn.metrics import roc_auc_score
from datasets import get_loaders
from models import resnet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

c:\Users\woojo\miniconda3\envs\panda1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = resnet.get_resnet_model(resnet_type=152)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
trainloader, testloader = get_loaders(dataset='cifar10', label_class=0, batch_size=32)


Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_feature_space = []
with torch.no_grad():
    for i in trainloader:
        imgs = i[0].to(device)
        _, features = model(imgs)
        train_feature_space.append(features)
    temp_feature_space = train_feature_space
    train_feature_space = torch.cat(train_feature_space, dim=0).contiguous().cpu().numpy()

In [6]:
temp_feature_space[0].shape

torch.Size([32, 2048])

In [7]:
train_feature_space.shape

(5000, 2048)

In [8]:
center = np.mean(train_feature_space, axis=0)

In [9]:
distances = np.linalg.norm(train_feature_space - center, axis=1)

In [10]:
avg_distance = np.mean(distances)

In [11]:
test_feature_space = []
with torch.no_grad():
    for i in testloader:
        imgs = i[0].to(device)
        _, features = model(imgs)
        test_feature_space.append(features)
    test_feature_space = torch.cat(test_feature_space, dim=0).contiguous().cpu().numpy()
    test_labels = testloader.dataset.targets

In [14]:
test_feature_space.shape

(10000, 2048)

In [16]:
test_distances = np.linalg.norm(test_feature_space - center, axis=1)

In [18]:
#normalize test_disatance by avg_distance
test_distances = test_distances / avg_distance

In [26]:
# index of anomaly in test_labels (list)
#change test_label list to numpy array
anomaly_index = np.where(np.array(test_labels)== 1)[0]
normal_index = np.where(np.array(test_labels)== 0)[0]

In [30]:
average_anomaly_distance = np.mean(test_distances[anomaly_index])
average_normal_distance = np.mean(test_distances[normal_index])

In [31]:
average_anomaly_distance

1.0258213

In [32]:
average_normal_distance

1.0253699

In [29]:
len(normal_index), len(anomaly_index)

(1000, 9000)

array([1.1838944 , 0.88959724, 0.8547148 , ..., 1.127255  , 1.0677854 ,
       0.9561146 ], dtype=float32)

In [ ]:
def get_avg_distance(model, device, trainloader, testloader):

    train_feature_space = []
    with torch.no_grad():
        for i in trainloader:
            imgs = i[0].to(device)
            _, features = model(imgs)
            train_feature_space.append(features)
        train_feature_space = torch.cat(train_feature_space, dim=0).contiguous().cpu().numpy()
    
    center = np.mean(train_feature_space, axis=0)
    distances = np.linalg.norm(train_feature_space - center, axis=1)
    avg_distance = np.mean(distances)


    return avg_distance

In [12]:
def get_score(model, device, trainloader, testloader):

    train_feature_space = []
    with torch.no_grad():
        for i in trainloader:
            imgs = i[0].to(device)
            _, features = model(imgs)
            train_feature_space.append(features)
        train_feature_space = torch.cat(train_feature_space, dim=0).contiguous().cpu().numpy()

    test_feature_space = []
    with torch.no_grad():
        for i in testloader:
            imgs = i[0].to(device)
            _, features = model(imgs)
            test_feature_space.append(features)
        test_feature_space = torch.cat(test_feature_space, dim=0).contiguous().cpu().numpy()
        test_labels = testloader.dataset.targets

    distances = utils.knn_score(train_feature_space, test_feature_space)

In [34]:
from collections import OrderedDict

In [35]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-5)

In [43]:
metrics = OrderedDict(lr=optimizer.param_groups[0]['lr'])

In [44]:
metrics

OrderedDict([('lr', 0.01)])

In [45]:
running_loss = OrderedDict([('compactness_loss',3.0)])

In [46]:
metrics.update([(k, v) for k, v in running_loss.items()])

In [47]:
metrics

OrderedDict([('lr', 0.01), ('compactness_loss', 3.0)])

In [48]:
metrics.update([("AUROC", 0.965)])

In [51]:
import logging

In [52]:
_logger = logging.getLogger('train')

In [53]:
_logger.info (metrics)